In [ ]:
!pip install openai


In [ ]:
import openai
openai.api_key = ''

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git sentencepiece


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-2gmvfab9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-2gmvfab9
  Resolved https://github.com/huggingface/transformers.git to commit c681b58b06f6fb8b5c331f380548af3b4b33f881
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch

In [ ]:
from transformers import SeamlessM4Tv2Model, AutoProcessor


# Load the processor and model
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large")

# Setup device
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tune

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

SeamlessM4Tv2Model(
  (shared): Embedding(256102, 1024, padding_idx=0)
  (text_encoder): SeamlessM4Tv2Encoder(
    (embed_tokens): SeamlessM4Tv2ScaledWordEmbedding(256102, 1024, padding_idx=0)
    (embed_positions): SeamlessM4Tv2SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-23): 24 x SeamlessM4Tv2EncoderLayer(
        (self_attn): SeamlessM4Tv2Attention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ffn): SeamlessM4Tv2FeedForwardNetwork(
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_fea

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset


In [ ]:
dataset = load_dataset("Ehtisham1328/urdu-idioms-with-english-translation", split='train')


Generating train split:   0%|          | 0/2111 [00:00<?, ? examples/s]

In [ ]:
print(dataset.features);

{'محنت کامیابی کی کنجی ہے۔': Value(dtype='string', id=None), 'Hard work is the key to success.': Value(dtype='string', id=None)}


In [ ]:
urdu_key = 'محنت کامیابی کی کنجی ہے۔'
english_key = 'Hard work is the key to success.'

In [ ]:
idioms = [(entry[urdu_key], entry[english_key]) for entry in dataset]

In [ ]:
translated_idioms = []
count = 0
for urdu_idiom, expected_english in idioms:
  if count == 500:
    break
  count+=1
  print(count)
  # Process the English idiom
  text_inputs = processor(text=urdu_idiom, src_lang="urd", return_tensors="pt").to(device)

  # Generate translation to english without generating speech
  output_tokens = model.generate(**text_inputs, tgt_lang="eng", generate_speech=False)
  #print(output_tokens[0].tolist())
  translated_text = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

  translated_idioms.append((urdu_idiom, translated_text, expected_english))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
del model

In [ ]:
del processor

In [52]:
import gc
gc.collect()

44

In [ ]:
from transformers import pipeline

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
translate_pipeline = pipeline(
  "text-generation",
  model=model_id,
  # model_kwargs={"torch_dtype": torch.bfloat16},
  # device_map="auto",
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def translate(text, pipeline):
  messages = [
      {"role": "system", "content": "You are a translator that is aware of idioms, colloquialisms, and figurative language. You take the input and Only output the English Translation. NOTHING ELSE!"},
      {"role": "user", "content": text}
  ]
  prompt = pipeline.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
  )

  # Define end-of-sentence terminators
  terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")  # Adjust or remove as necessary
  ]

  # Generate output with specific control on sampling
  results = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
  )
  # prompt = f"Translate this to English: {text}"
  # results = pipeline(messages, max_length=60, num_return_sequences=1)
  translated_text = results[0]["generated_text"][len(prompt):]
  print(translated_text)
  return translated_text

In [ ]:
translated_idioms_llama = []
count = 0

for urdu_idiom, expected_english in idioms:
  if count == 500:
    break
  count+=1
  print(count)
  translated_text = translate(urdu_idiom, translate_pipeline)
  translated_idioms_llama.append((urdu_idiom, translated_text, expected_english))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Courage of men is the help of God."
2


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Where's my courage?
498


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


To and fro, to no avail.
499


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


To cut a long story short.
500
A wolf in sheep's clothing.


In [54]:
import openai
openai.api_key = ''

In [79]:
def translate_GPT(urdu_idiom):
    prompt = f"Translate this Urdu idiom into English, considering both its literal and figurative meaning: '{urdu_idiom}'. Only output the Translation, NO comments or notes."
    response = openai.chat.completions.create(
      model="gpt-4",  # Using the GPT-4 model
      messages=[
          {"role": "system", "content": "You are a highly skilled translator with expertise in Urdu to English translation, aware of both literal and figurative language."},
          {"role": "user", "content": urdu_idiom}
      ],
      max_tokens=60,
      temperature=0.5
    )
    return response.choices[0].message.content


In [80]:
translated_idioms_GPT4 = []
count = 0

for urdu_idiom, expected_english in idioms:
  if count == 500:
    break
  count+=1
  print(count)
  translated_text = translate_GPT(urdu_idiom)
  print(translated_text)
  translated_idioms_GPT4.append((urdu_idiom, translated_text, expected_english))

1
The courage of a man is the help of God.
2
There's no work without cost.
3
Practice makes a man perfect.
4
Where there's a will, there's a way.
5
If you are good, then everything is good.
6
Life is not a bed of roses.
7
Life is the name of vivacity.
8
Experience is the greatest teacher.
9
Once bitten, twice shy.
10
Time waits for no one.
11
A stitch in time saves nine.
12
Time is the greatest healer.
13
The tune of time is always good.
14
When the time comes, even a donkey is made a father.
15
"Better late than never."
16
Don't put off today's work till tomorrow.
17
Love is blind.
18
In love and war, everything is fair.
19
The heart finds its way to another heart.
20
When the heart finds peace, everyone understands celebration.
21
A friend is one who comes in handy during adversity.
22
Each to their own.
23
Not everyone who shakes hands is a friend.
24
The enemy of a friend is an enemy, the enemy of an enemy is a friend.
25
The friendship remains alive, my friend.
26
The prayer of a 

In [2]:
import csv

In [ ]:
filename = 'translations_SeamlessM4T_500_urdu.csv'

# Writing to a CSV file
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['Urdu Idiom', 'Model Output', 'Correct Translation'])
    # Write each translation group
    for translation in translated_idioms:
        writer.writerow(translation)

print(f"Data successfully written to {filename}")

Data successfully written to translations_SeamlessM4T_500_urdu.csv


In [ ]:
filename = 'translations_LlamaV3_8B_Instruct_500_urdu.csv'

# Writing to a CSV file
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['Urdu Idiom', 'Model Output', 'Correct Translation'])
    # Write each translation group
    for translation in translated_idioms_llama:
        writer.writerow(translation)

print(f"Data successfully written to {filename}")

Data successfully written to translations_LlamaV3_8B_Instruct_500_urdu.csv


In [81]:
filename = 'translations_GPT4_500_urdu.csv'

# Writing to a CSV file
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['Urdu Idiom', 'Model Output', 'Correct Translation'])
    # Write each translation group
    for translation in translated_idioms_GPT4:
        writer.writerow(translation)

print(f"Data successfully written to {filename}")

Data successfully written to translations_GPT4_500_urdu.csv


In [4]:
filename = 'translations_SeamlessM4T_500_urdu.csv'
translated_idioms = []
# Reading from a CSV file
with open(filename, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    # Skip the header
    next(reader)
    # Read each row and add it to the translations list as a tuple
    for row in reader:
        translated_idioms.append(tuple(row))

# Print the translations array to verify
print(len(translated_idioms))
for translation in translated_idioms:
    print(translation)

500
('ہمتِ مرداں مددِ خدا۔', 'We are dead, God help us.', 'The courage of men is aided by God.')
('کام بِن دام نہیں۔', "It's not a job.", 'Nothing comes without a price.')
('مشق انسان کو کامل بنا دیتی ہے۔', 'Practice makes perfect.', 'Practice makes a person perfect.')
('جہاں چاہ وہاں راہ۔', 'Wherever there is a way.', 'Where there is a will, there is a way.')
('انت بھلا تو سب بھلا۔', 'All is well.', 'If the end is good, everything is good.')
('زندگی پھولوں کی سیج نہیں۔', 'Life is not a bouquet of flowers.', 'Life is not a bed of roses.')
('زندگی زندہ دلی کا نام ہے۔', 'Life is the name of the living heart.', 'Life is the name of a lively heart.')
('تجربہ سب سے بڑا استاد ہے۔', 'Experience is the greatest teacher.', 'Experience is the greatest teacher.')
('دودھ کا جلا چھاچھ بھی پھونک پھونک کر پیتا ہے۔', 'Milk is also blown away by the wind.', 'Even a burnt milk pudding is consumed bit by bit.')
('وقت کسی کا انتظار نہیں کرتا۔', 'Time does not wait for anyone.', 'Time waits for no one.')
(

In [5]:
filename = 'translations_LlamaV3_8B_Instruct_500_urdu.csv'
translated_idioms_llama = []
# Reading from a CSV file
with open(filename, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    # Skip the header
    next(reader)
    # Read each row and add it to the translations list as a tuple
    for row in reader:
        translated_idioms_llama.append(tuple(row))

# Print the translations array to verify
print(len(translated_idioms_llama))
for translation in translated_idioms_llama:
    print(translation)

500
('ہمتِ مرداں مددِ خدا۔', '"Courage of men is the help of God."', 'The courage of men is aided by God.')
('کام بِن دام نہیں۔', "It's all or nothing.", 'Nothing comes without a price.')
('مشق انسان کو کامل بنا دیتی ہے۔', "A person's makeup is complete.", 'Practice makes a person perfect.')
('جہاں چاہ وہاں راہ۔', "Where there's a will, there's a way.", 'Where there is a will, there is a way.')
('انت بھلا تو سب بھلا۔', "You're as good as everyone else.", 'If the end is good, everything is good.')
('زندگی پھولوں کی سیج نہیں۔', 'Life is not a bed of roses.', 'Life is not a bed of roses.')
('زندگی زندہ دلی کا نام ہے۔', "Life is the name of the heart's desire.", 'Life is the name of a lively heart.')
('تجربہ سب سے بڑا استاد ہے۔', 'Experience is the best teacher.', 'Experience is the greatest teacher.')
('دودھ کا جلا چھاچھ بھی پھونک پھونک کر پیتا ہے۔', "He's drinking his tea with a lot of sugar and noise.", 'Even a burnt milk pudding is consumed bit by bit.')
('وقت کسی کا انتظار نہیں کرتا۔'

In [6]:
filename = 'translations_GPT4_500_urdu.csv'
translated_idioms_GPT4 = []
# Reading from a CSV file
with open(filename, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    # Skip the header
    next(reader)
    # Read each row and add it to the translations list as a tuple
    for row in reader:
        translated_idioms_GPT4.append(tuple(row))

# Print the translations array to verify
print(len(translated_idioms_GPT4))
for translation in translated_idioms_GPT4:
    print(translation)

500
('ہمتِ مرداں مددِ خدا۔', 'The courage of a man is the help of God.', 'The courage of men is aided by God.')
('کام بِن دام نہیں۔', "There's no work without cost.", 'Nothing comes without a price.')
('مشق انسان کو کامل بنا دیتی ہے۔', 'Practice makes a man perfect.', 'Practice makes a person perfect.')
('جہاں چاہ وہاں راہ۔', "Where there's a will, there's a way.", 'Where there is a will, there is a way.')
('انت بھلا تو سب بھلا۔', 'If you are good, then everything is good.', 'If the end is good, everything is good.')
('زندگی پھولوں کی سیج نہیں۔', 'Life is not a bed of roses.', 'Life is not a bed of roses.')
('زندگی زندہ دلی کا نام ہے۔', 'Life is the name of vivacity.', 'Life is the name of a lively heart.')
('تجربہ سب سے بڑا استاد ہے۔', 'Experience is the greatest teacher.', 'Experience is the greatest teacher.')
('دودھ کا جلا چھاچھ بھی پھونک پھونک کر پیتا ہے۔', 'Once bitten, twice shy.', 'Even a burnt milk pudding is consumed bit by bit.')
('وقت کسی کا انتظار نہیں کرتا۔', 'Time waits 

In [7]:
from nltk.translate.bleu_score import sentence_bleu

In [8]:
!pip install nltk
import nltk
from nltk.translate.bleu_score import corpus_bleu
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
def calculate_bleu(translated_idioms):
  references = []
  candidates = []
  for _, translated_text, expected_english in translated_idioms:
    # Tokenize the expected and generated translations
    reference = nltk.word_tokenize(expected_english.lower())
    candidate = nltk.word_tokenize(translated_text.lower())

    # For corpus BLEU scores, references are lists of lists, candidates are lists
    references.append([reference])  # reference needs to be a list of lists
    candidates.append(candidate)

  # Calculate the BLEU score
  bleu_score = corpus_bleu(references, candidates)
  return bleu_score

def calculate_exact_match_score(translated_idioms):
  exact_matches = sum(1 for _, translated, expected in translated_idioms if translated.strip().lower() == expected.strip().lower())
  return exact_matches / len(translated_idioms)

def weighted_score(bleu_score, exact_match_score, bleu_weight=0.5, exact_match_weight=0.5):
  return (bleu_score * bleu_weight) + (exact_match_score * exact_match_weight)

In [10]:
print(translated_idioms)

[('ہمتِ مرداں مددِ خدا۔', 'We are dead, God help us.', 'The courage of men is aided by God.'), ('کام بِن دام نہیں۔', "It's not a job.", 'Nothing comes without a price.'), ('مشق انسان کو کامل بنا دیتی ہے۔', 'Practice makes perfect.', 'Practice makes a person perfect.'), ('جہاں چاہ وہاں راہ۔', 'Wherever there is a way.', 'Where there is a will, there is a way.'), ('انت بھلا تو سب بھلا۔', 'All is well.', 'If the end is good, everything is good.'), ('زندگی پھولوں کی سیج نہیں۔', 'Life is not a bouquet of flowers.', 'Life is not a bed of roses.'), ('زندگی زندہ دلی کا نام ہے۔', 'Life is the name of the living heart.', 'Life is the name of a lively heart.'), ('تجربہ سب سے بڑا استاد ہے۔', 'Experience is the greatest teacher.', 'Experience is the greatest teacher.'), ('دودھ کا جلا چھاچھ بھی پھونک پھونک کر پیتا ہے۔', 'Milk is also blown away by the wind.', 'Even a burnt milk pudding is consumed bit by bit.'), ('وقت کسی کا انتظار نہیں کرتا۔', 'Time does not wait for anyone.', 'Time waits for no on

In [11]:
print(translated_idioms_llama)

[('ہمتِ مرداں مددِ خدا۔', '"Courage of men is the help of God."', 'The courage of men is aided by God.'), ('کام بِن دام نہیں۔', "It's all or nothing.", 'Nothing comes without a price.'), ('مشق انسان کو کامل بنا دیتی ہے۔', "A person's makeup is complete.", 'Practice makes a person perfect.'), ('جہاں چاہ وہاں راہ۔', "Where there's a will, there's a way.", 'Where there is a will, there is a way.'), ('انت بھلا تو سب بھلا۔', "You're as good as everyone else.", 'If the end is good, everything is good.'), ('زندگی پھولوں کی سیج نہیں۔', 'Life is not a bed of roses.', 'Life is not a bed of roses.'), ('زندگی زندہ دلی کا نام ہے۔', "Life is the name of the heart's desire.", 'Life is the name of a lively heart.'), ('تجربہ سب سے بڑا استاد ہے۔', 'Experience is the best teacher.', 'Experience is the greatest teacher.'), ('دودھ کا جلا چھاچھ بھی پھونک پھونک کر پیتا ہے۔', "He's drinking his tea with a lot of sugar and noise.", 'Even a burnt milk pudding is consumed bit by bit.'), ('وقت کسی کا انتظار نہیں 

In [12]:
print(translated_idioms_GPT4)

[('ہمتِ مرداں مددِ خدا۔', 'The courage of a man is the help of God.', 'The courage of men is aided by God.'), ('کام بِن دام نہیں۔', "There's no work without cost.", 'Nothing comes without a price.'), ('مشق انسان کو کامل بنا دیتی ہے۔', 'Practice makes a man perfect.', 'Practice makes a person perfect.'), ('جہاں چاہ وہاں راہ۔', "Where there's a will, there's a way.", 'Where there is a will, there is a way.'), ('انت بھلا تو سب بھلا۔', 'If you are good, then everything is good.', 'If the end is good, everything is good.'), ('زندگی پھولوں کی سیج نہیں۔', 'Life is not a bed of roses.', 'Life is not a bed of roses.'), ('زندگی زندہ دلی کا نام ہے۔', 'Life is the name of vivacity.', 'Life is the name of a lively heart.'), ('تجربہ سب سے بڑا استاد ہے۔', 'Experience is the greatest teacher.', 'Experience is the greatest teacher.'), ('دودھ کا جلا چھاچھ بھی پھونک پھونک کر پیتا ہے۔', 'Once bitten, twice shy.', 'Even a burnt milk pudding is consumed bit by bit.'), ('وقت کسی کا انتظار نہیں کرتا۔', 'Time 

In [16]:
bleu_score = calculate_bleu(translated_idioms)
exact_match_score = calculate_exact_match_score(translated_idioms)
final_score = weighted_score(bleu_score, exact_match_score, bleu_weight=0.5, exact_match_weight=0.5)
print(f"Combined Score: {final_score:.2f}")

Combined Score: 0.05


In [17]:
bleu_score_llama = calculate_bleu(translated_idioms_llama)
exact_match_score_llama = calculate_exact_match_score(translated_idioms_llama)
final_score_llama = weighted_score(bleu_score_llama, exact_match_score_llama, bleu_weight=0.5, exact_match_weight=0.5)
print(f"Combined Score: {final_score_llama:.2f}")

Combined Score: 0.04


In [18]:
bleu_score_GPT4 = calculate_bleu(translated_idioms_GPT4)
exact_match_score_GPT4 = calculate_exact_match_score(translated_idioms_GPT4)
final_score_GPT4 = weighted_score(bleu_score_GPT4, exact_match_score_GPT4, bleu_weight=0.5, exact_match_weight=0.5)
print(f"Combined Score: {final_score_GPT4:.2f}")

Combined Score: 0.05


use difference between embeddings to calculate accuracy of translations

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine

# Load sentence embedding model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(1)

def calculate_similarity(embedding1, embedding2):
    # Detach embeddings from the computation graph, ensure they are 1-D, and convert to numpy arrays
    emb1 = embedding1.detach().squeeze().numpy()
    emb2 = embedding2.detach().squeeze().numpy()
    return 1 - cosine(emb1, emb2)

def evaluate_translations(translations):
    total_cosine_similarity = 0
    total_exact_match = 0
    num_samples = len(translations)

    for urdu_idiom, model_output, correct_translation in translations:
        # Get embeddings
        model_output_embedding = get_embedding(model_output)
        correct_translation_embedding = get_embedding(correct_translation)

        # Calculate cosine similarity
        cosine_sim = calculate_similarity(model_output_embedding, correct_translation_embedding)
        total_cosine_similarity += cosine_sim

        # Calculate exact match
        exact_match = 1 if model_output.strip().lower() == correct_translation.strip().lower() else 0
        total_exact_match += exact_match

    # Calculate weighted average (adjust weights as needed)
    average_cosine_similarity = total_cosine_similarity / num_samples
    average_exact_match = total_exact_match / num_samples
    weighted_average = 0.7 * average_cosine_similarity + 0.3 * average_exact_match

    return {
        "average_cosine_similarity": average_cosine_similarity,
        "average_exact_match": average_exact_match,
        "weighted_average": weighted_average
    }

In [ ]:
results = evaluate_translations(translated_idioms)
print("Average Cosine Similarity:", results["average_cosine_similarity"])
print("Average Exact Match Score:", results["average_exact_match"])
print("Weighted Average Score:", results["weighted_average"])

Average Cosine Similarity: 0.5486460798829794
Average Exact Match Score: 0.012
Weighted Average Score: 0.38765225591808555


In [ ]:
results_llama = evaluate_translations(translated_idioms_llama)
print("Average Cosine Similarity:", results_llama["average_cosine_similarity"])
print("Average Exact Match Score:", results_llama["average_exact_match"])
print("Weighted Average Score:", results_llama["weighted_average"])

Average Cosine Similarity: 0.5282049338370561
Average Exact Match Score: 0.012
Weighted Average Score: 0.37334345368593924


In [85]:
results_GPT4 = evaluate_translations(translated_idioms_GPT4)
print("Average Cosine Similarity:", results_GPT4["average_cosine_similarity"])
print("Average Exact Match Score:", results_GPT4["average_exact_match"])
print("Weighted Average Score:", results_GPT4["weighted_average"])

Average Cosine Similarity: 0.4799986736625433
Average Exact Match Score: 0.022
Weighted Average Score: 0.3425990715637803


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('all-mpnet-base-v2')

def calculate_similarity_st(embedding1, embedding2):
    return util.pytorch_cos_sim(embedding1, embedding2).item()

def evaluate_translations_st(translations):
    average_cosine_similarity = 0
    exact_match_count = 0
    total = len(translations)

    for urdu_idiom, model_output, correct_translation in translations:
        model_output_embedding = model.encode(model_output, convert_to_tensor=True)
        correct_translation_embedding = model.encode(correct_translation, convert_to_tensor=True)

        similarity = calculate_similarity_st(model_output_embedding, correct_translation_embedding)
        average_cosine_similarity += similarity

        if model_output.strip().lower() == correct_translation.strip().lower():
            exact_match_count += 1

    average_cosine_similarity /= total
    exact_match_score = exact_match_count / total
    weighted_average = 0.7 * average_cosine_similarity + 0.3 * exact_match_score

    return {
        "average_cosine_similarity": average_cosine_similarity,
        "average_exact_match": exact_match_score,
        "weighted_average": weighted_average
    }

In [ ]:
results = evaluate_translations_st(translated_idioms)
print("Average Cosine Similarity:", results["average_cosine_similarity"])
print("Average Exact Match Score:", results["average_exact_match"])
print("Weighted Average Score:", results["weighted_average"])

Average Cosine Similarity: 0.39130998904258013
Average Exact Match Score: 0.012
Weighted Average Score: 0.2775169923298061


In [ ]:
results_llama = evaluate_translations_st(translated_idioms_llama)
print("Average Cosine Similarity:", results_llama["average_cosine_similarity"])
print("Average Exact Match Score:", results_llama["average_exact_match"])
print("Weighted Average Score:", results_llama["weighted_average"])

Average Cosine Similarity: 0.35184686138294635
Average Exact Match Score: 0.012
Weighted Average Score: 0.24989280296806243


In [86]:
results_GPT4_st = evaluate_translations_st(translated_idioms_GPT4)
print("Average Cosine Similarity:", results_GPT4_st["average_cosine_similarity"])
print("Average Exact Match Score:", results_GPT4_st["average_exact_match"])
print("Weighted Average Score:", results_GPT4_st["weighted_average"])

Average Cosine Similarity: 0.4799986736625433
Average Exact Match Score: 0.022
Weighted Average Score: 0.3425990715637803
